In [1]:
import pandas as pd
import re
from nltk import word_tokenize

In [2]:
df = pd.read_excel('address_rd_split (copy).xlsx')

In [3]:
address = df.values.tolist()

In [4]:
def clean_address(address):
    typo = {"Meanchey":"Mean Chey",
            "Kamdal":"Kandal",
            "Sihanoukville": "Sihanouk",
            "Chamkarmon": "Chamkar Mon",
            "Village": "",
            "Commune": "",
            "District": "",
            "Province": "",
            "Beoung": "Boeung",
            "Sangkat": "", 
            "Tonle Bassac": "Tonle Basak",
            "Por Sen Chey": "Por Senchey",
            "Ponhea Leu": "Ponhea Lueu",
            "Oulampik": "Olympic",
            "7 Makara": "Prampir Meakkakra"} 

    abbr = {"BKK" : "Boeng Keng Kang",
            "\bTK\b": "Toul Kork",
            " Str ": " Street ",
            " st ":" Street ",}                    


    cleaning = {re.compile('\\-+|\\:+|\\$+'): '',
                re.compile('\\b(\\d+)(\\s+\\1\\b)+'): ', ',
                re.compile('\\(|\\)'): '',
                re.compile(',+\s+,+'): ',',
                re.compile('\s+,\s+'): ', ',
                re.compile('^,|,$'): '',
                re.compile('\s+'): ' ',
                re.compile('^\s+|\s+$'): '',
                re.compile('`'): '\''}    

    pattern = '|'.join(sorted(re.escape(k) for k in typo))    
    pattern_1 = '|'.join(sorted(re.escape(k) for k in abbr)) 

    def multiple_replace(myDict, text):
        for rx,repl in myDict.items():
            text = rx.sub(repl, text)
        return text

    if len(address) == 1:
        address1 = [re.sub(pattern, lambda m: typo.get(m.group(0)), x, flags=re.IGNORECASE) for x in address]
        address2 = [re.sub(pattern_1, lambda m: abbr.get(m.group(0)), x, flags=re.IGNORECASE) for x in address1]
        address3 = [multiple_replace(cleaning, x) for x in address2]
        return address3

    else:

        address1 = [[re.sub(pattern, lambda m: typo.get(m.group(0)), x, flags=re.IGNORECASE) for x in l] for l in address]
        address2 = [[re.sub(pattern_1, lambda m: abbr.get(m.group(0)), x, flags=re.IGNORECASE) for x in l] for l in address1]
        address3 = [[multiple_replace(cleaning, x) for x in l] for l in address2]

        return address3          
             

In [5]:
address_clean = clean_address(address)

In [6]:
ls = []
for x in address_clean:
    for i in x:
        tk = word_tokenize(i)
    ls.append(tk)

In [7]:
cl = pd.DataFrame({"address": [(" ".join(x)) for x in address_clean],
                "tokenize": [(" ".join(x)) for x in ls]})                

In [8]:
cl_1 = cl['tokenize'].str.split(expand=True)

In [9]:
final = cl.join(cl_1)

In [10]:
final = final.drop(['tokenize'], axis=1)

In [11]:
# final.to_excel('final_clean.xlsx', index=False)

In [12]:
street = ['Corner Street 13 & 102, SangKat Wat Phnom, Khan Daun Penh, Phnom Penh',
'Corner Street 13 & 102, SangKat Wat Phnom, Khan Daun Penh, Phnom Penh', 
'Corner Street 13 & 102, Sang Kat Wat Phnom, Khan Daun Penh, Phnom Penh',
'# 488 BC Eo, St. Moev Seng Toung, Sangkat Toek la ork I, Khan Toul kork, Phnom Penh',
'Russian Bola vat ,Sangkat Ka Kap, Khan Por Sen Chey, Phnom Penh',
'# 217, St. Monica Ray, Sangkat Boeng Salang, Tuol Kork, Phnom Penh',
'# 115. 113, Street 199, Sangkat Toul Svay Prey II, Khan Chamkamorn, Phnom Penh', 
'# 09 C, National Highway 1, Village Boeung Chhouk, Sangkat Niroth, Khan Chbar am pov,Phnom Penh',
'# 289, St. Ville (274), Sangkat Veal Vong, Khan 7 Makara, Phnom Penh',
'Phnom Penh International Airport, Russian Federation, Kakap Sangkat, Khan Por Sen Chey, Phnom Penh',
'# 528, Group 4, National Highway 5, Sangkat Chrang Chamres 1, Khan Russey Keo, Phnom Penh',
'National Highway 6 ,Village 6, Sangkat Kampong Thom, Krung Stoeng Sen City, Kampong Thom Province',
'2nd Street, village center, Sangkat Kampong Pranak, Kroung Preah Vihear, Preah Vihear Province',
'St. Poukambor, Sangkat Svay Dongkom, Krong Siem Reap (17252), Siem Reap Province',
'14 villages, Khemarakphumin, Sangkat Kampong Cham, Krung Kampong Cham, Kampong Cham Province',
'Road 1, Mephleung Village, Sangkat Svay Rieng, Svay Rieng City, Svay Rieng Province',
'Route 1 (Reuters river), 2 Village, Sangkat Phtas Prey, Krung Pursat , Pursat Province',
'1, Sangkat Khsam, Krung Kampong Chhnang, Kampong Chhnang Province',
'Street Corner 105 & 206, Takmao Village, Sangkat Takmao,Krung Takmao, Kandal', 
'Village 3, Sangkat Kampong Leav, Krung Prey Veng, Prey Vengn Province', 
'Village 3, Sangkat Preah Ponlea,Krung Serey Sophorn, Banteay Meanchey Province', 
'Route 1, Kamakor Village, Sangkat Svaypor, Krunng Battambang, Battambang Province',
'Route 12, Kratie Village, Sangkat Kratie, Krung Kratie, Kratie (opposite the roundabout Dragon Fountain)',
'Road No. 78, Phum Apivath, Sangkat Labarnseik, Krung Banlung, Ratanakiri Province',
'Phum Prek, Street 63, Sangkat Stung Treng, Krung Stung Treng, Stung Treng Province', 
'Route 735 (along the creek), Phum May 1, Krung Kampot, Kampot Province',
'Corner road 22A & 2316, Phum Kep, Sangkat Kep, Krung Kep, Kep Province',
'1-Phsar Ler, Sangkat 2,Krung Preah Sihanouk, Phreah Sihanouk Province',
'2- Krung 27, Phum 3,Sangkat 3, KrungPreah Sihanouk, Preah Sihanouk Province',
'Village 1, Sangkat Smach Meanchey, Krung Khemrakphumin, Koh Kong Province',
'Phum Wat, Sangkat Pailin, Krung Pailin, Pailin Province',
'Chhouk Village, Sangkat Samrong, Krung Samrong, Oddarmeanchey Province',
'Phum Pi, Sangkat Rorkaknoung, Krung Dounkeo, Takeo Province',
'Phum Daohkramoum, Sangkat Sokhadom, Krung Senmonorom, Mondulkiri Province',
'Road 4, Phum Samnoeng, Sangkat Rokarthom, Krung Chbamorn, Kampong Speu Provice',
'Phum Soungkert, Sangkat Soung, Krung Suong. Thbungkhmum Province']

In [13]:
str_ls = []
for x in street:
    str_ls.append(word_tokenize(x))

In [14]:
old_str = []
for x in street:
    x_ls = [x]
    old_str.append(x_ls)
old_str

[['Corner Street 13 & 102, SangKat Wat Phnom, Khan Daun Penh, Phnom Penh'],
 ['Corner Street 13 & 102, SangKat Wat Phnom, Khan Daun Penh, Phnom Penh'],
 ['Corner Street 13 & 102, Sang Kat Wat Phnom, Khan Daun Penh, Phnom Penh'],
 ['# 488 BC Eo, St. Moev Seng Toung, Sangkat Toek la ork I, Khan Toul kork, Phnom Penh'],
 ['Russian Bola vat ,Sangkat Ka Kap, Khan Por Sen Chey, Phnom Penh'],
 ['# 217, St. Monica Ray, Sangkat Boeng Salang, Tuol Kork, Phnom Penh'],
 ['# 115. 113, Street 199, Sangkat Toul Svay Prey II, Khan Chamkamorn, Phnom Penh'],
 ['# 09 C, National Highway 1, Village Boeung Chhouk, Sangkat Niroth, Khan Chbar am pov,Phnom Penh'],
 ['# 289, St. Ville (274), Sangkat Veal Vong, Khan 7 Makara, Phnom Penh'],
 ['Phnom Penh International Airport, Russian Federation, Kakap Sangkat, Khan Por Sen Chey, Phnom Penh'],
 ['# 528, Group 4, National Highway 5, Sangkat Chrang Chamres 1, Khan Russey Keo, Phnom Penh'],
 ['National Highway 6 ,Village 6, Sangkat Kampong Thom, Krung Stoeng Sen Ci

In [15]:
new_street = pd.DataFrame({"address": [(" ".join(x)) for x in old_str],
                "tokenize": [(" ".join(x)) for x in str_ls]})                

In [16]:
cl_2 = new_street['tokenize'].str.split(expand=True)

In [17]:
# cl_2.to_excel('lol.xlsx', index=False)

In [18]:
clean_address(['# 488 BC Eo, St. Moev Seng Toung, Sangkat Toek la ork I, Khan Toul kork, Phnom Penh'])

['# 488 BC Eo, St. Moev Seng Toung, Toek la ork I, Khan Toul kork, Phnom Penh']

In [23]:
word_tokenize('Olympia City C4, 217, Veal Vong, Khan Prampir Meakkakra, Russey Keo, Russey Keo, Phnom Penh')

['Olympia',
 'City',
 'C4',
 ',',
 '217',
 ',',
 'Veal',
 'Vong',
 ',',
 'Khan',
 'Prampir',
 'Meakkakra',
 ',',
 'Russey',
 'Keo',
 ',',
 'Russey',
 'Keo',
 ',',
 'Phnom',
 'Penh']